# Python Lab 02: Orthogonalization Methods
## Francesco Della Santa, Computational Linear Algebra for Large Scale Problems, Politecnico di Torino

In this lesson, we will implement the (Modified) *Gram-Schmidt* method, the *Givens* method and the *House-
holder* method.

In [1]:
# ***** ATTENTION! *****
# If you want that the "%matplotlib widget" works, you need the package ipympl (pip install ipympl)
#
#
# MATPLOTLIB INTERACTIVE VISUALIZATION. REMOVE (OR COMMENT) IF YOU NEED TO PRINT THE NOTEBOOK AS A PDF, SOMETIMES IT DOES NOT WORK WELL...
%matplotlib widget
#
#

from IPython.display import display  # to display variables in a "nice" way
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from testmatrices import A1, b1, Ab1

## The Gram-Schmidt Method

Given an invertible square matrix $X\in\mathbb{R}^{n\times n}$, the Gram-Schmidt (GS) method compute two matrices, $Q, R \in\mathbb{R}^{n\times n}$, such that:
- $Q$ is orthogonal (i.e., $QQ^\top = \mathbb{I}_n = Q^{\top}Q$);
- The columns $\{\boldsymbol{q}_1,\ldots ,\boldsymbol{q}_n\}$ of $Q$ are an orthonormal base of the space $\langle \boldsymbol{x}_1,\ldots ,\boldsymbol{x}_n\rangle$ generated by the columns of $X$ (obvious if $X$ is square, but not if we considere the "extended case" $X\in\mathbb{R}^{m\times n}$, $m > n$);
- $R\in\mathbb{R}^{n\times n}$ upper triangular;
- $QR=X$.


#### Description of the GS Method

The idea of the GS method is to *build* iteratively the columns of $Q$ from the columns of $X$. In particular, we compute the column $\boldsymbol{q}_j$ as the column $\boldsymbol{x}_j$ from which we subtract the previous columns of $Q$ multiplied by their projection onto $\boldsymbol{x}_j$ (because we want orthogonal columns for $Q$); then, we normalize the result of this operation. The element $r_{ij}$, with $i< j$, are the projections of $\boldsymbol{q}_i$ on $\boldsymbol{x}_j$, while $r_{jj}$ is the norm of $\boldsymbol{q}_j$ *before the normalization step*.

From a "pseudocode point-of-view":
1. **for** $j=1,\ldots , n$ **do:**
    1. $\boldsymbol{q}_j\gets \boldsymbol{x}_j - \sum_{i=1}^{j-1}(\boldsymbol{x}_j,\boldsymbol{q}_i) \boldsymbol{q}_i$, where$(\boldsymbol{x}_j,\boldsymbol{q}_i)$ becomes $r_{ij}$;
    1. $\boldsymbol{q}_j\gets \frac{\boldsymbol{q}_j}{||\boldsymbol{q}_j||}$, where $||\boldsymbol{q}_j||$ becomes $r_{jj}$.

#### Exercise 1: GS Method

Complete the function in the following cell, such that it performs the GS method for any square matrix. The function must return not only $Q$ and $R$, but also the following norms representing the quality of the outputs:
- $||\mathbb{I}_n - QQ^\top||$;
- $||Q^\top X - R||$.

**Suggestions:** look at the help of the function np.*linalg*.**norm** to compute the norm of an array.

In [2]:
def gramschmidt(X):
    """
    Function that performs the Gram-Schmidt method for a given square Matrix (changing the code is generalizable to
    rectangular matrices)
    The matrix must have full rank.
    :param X: square matrix represented as 2D-array object (numpy ndarray);
    :return Q: 2D-array (orthogonal matrix) 
    :return R: 2D-array (upper triangle matrix).
    :return Qqual: norm ||In - Q @ Q.T||
    :return QRqual: norm ||Q.T @ X - R||
    """

    m, n = X.shape
    
    if m != n:
        print('MATRIX IS NOT SQUARE!')
        return None, None, None, None
    
    # Initialization of the matrices
    R = np.zeros((n,n)) 
    Q = np.zeros((n,n)) 

    R[0, 0] = np.linalg.norm(X[:,0])  
    Q[:, 0] = X[:, 0].copy() / R[0,0]

    for j in range(1, n):
        Q[:,j] = X[:,j].copy()
        for i in range(1, j):
            # proiezione di j su i 
            R[i,j] = X[:,j] @ Q[:,i]
            Q[:, j] -= R[i,j] * Q[:,i]
            
        R[j, j] = np.linalg.norm(Q[:,j])
        Q[:,j] = Q[:,j]/R[j,j]
        
    Qqual = np.linalg.norm(np.identity(n) - Q @ Q.T) 
    
    QRqual = np.linalg.norm(Q.T @ X - R)  

    return Q, R, Qqual, QRqual

In [3]:
# GRAM-SCHMIDT METHOD TEST

print('********** Running GS on A1...')
Q, R, Qqual, QRqual = gramschmidt(A1)
print('')

print('********** Matrix Q of A1 **********')
print(Q)
print('')
print(f'********** "Orthogonal Quality" of Q: {Qqual}')
print('')

print('********** Matrix R of A1 **********')
print(R)
print('')
print(f'********** "Factorization Quality" of Q @ R: {QRqual}')
print('')

print('********** Running GS on Ab1...')
_, _, _, _ = gramschmidt(Ab1)

********** Running GS on A1...

********** Matrix Q of A1 **********
[[ 0.49493563  0.27430061  0.67086781 -0.58353431]
 [ 0.10652533  0.02191507  0.71190709  0.59196435]
 [ 0.4727903   0.55434307 -0.11694396  0.52489343]
 [ 0.72122147  0.78548244 -0.17160648 -0.18317426]]

********** "Orthogonal Quality" of Q: 1.4134220179061285

********** Matrix R of A1 **********
[[1.09905346 0.         0.         0.        ]
 [0.         1.18225488 0.37028917 0.93376472]
 [0.         0.         0.9499728  0.6152959 ]
 [0.         0.         0.         0.66697111]]

********** "Factorization Quality" of Q @ R: 1.9455544728877086

********** Running GS on Ab1...
MATRIX IS NOT SQUARE!


### The Modified GS Method

The **Modified Gram-Schmidt** method is introduced for numerical applications because, in finite arithmetic, it is more stable than the classic method.

In exact arithmetic, the Modified GS method returns the same result of the GS method.

The Modified GS method, changes the operations in this way:
1. **for** $j=1,\ldots , n$ **do:**
    1. $\boldsymbol{q}_j\gets \boldsymbol{q}_j - (\boldsymbol{q}_j,\boldsymbol{q}_i) \boldsymbol{q}_i$, iteratively for each $i=1,\ldots, j-1$, where$(\boldsymbol{q}_j,\boldsymbol{q}_i)$ becomes $r_{ij}$;
    1. $\boldsymbol{q}_j\gets \frac{\boldsymbol{q}_j}{||\boldsymbol{q}_j||}$, where $||\boldsymbol{q}_j||$ becomes $r_{jj}$.

#### Exercise 2: Modified GS Method

Complete the function in the following cell, such that it performs the Modified GS method for any square matrix.

In [4]:
def mod_gramschmidt(X):
    """
    Function that performs the Modified Gram-Schmidt method for a given square Matrix (changing the code is
    generalizable to rectangular matrices)
    The matrix must have full rank.
    :param X: square matrix represented as 2D-array object (numpy ndarray);
    :return Q: 2D-array (orthogonal matrix) 
    :return R: 2D-array (upper triangle).
    :return Qqual: norm ||In - Q @ Q.T||
    :return QRqual: norm ||Q.T @ X - R||
    """

    m, n = X.shape
    
    if m != n:
        print('MATRIX IS NOT SQUARE!')
        return None, None, None, None
    
    R = np.zeros((n,n))  
    Q = np.zeros((n,n))  

    R[0, 0] = np.linalg.norm(X[:,0]) 
    Q[:, 0] = X[:, 0].copy() / R[0,0] 

    for j in range(1, n):
        Q[:,j] = X[:,j].copy()
        for i in range(1, j):
            # proiezione di j su i 
            R[i,j] = Q[:,j] @ Q[:,i]
            Q[:, j] -= (R[i,j])*Q[:,i]
            
        R[j, j] = np.linalg.norm(Q[:,j])
        Q[:,j] = Q[:,j]/R[j,j]
        
    Qqual = np.linalg.norm(np.identity(n) - Q @ Q.T)  
    
    QRqual = np.linalg.norm(Q.T @ X - R)  # <-- TODO! 

    return Q, R, Qqual, QRqual

In [5]:
# MODIFIED GRAM-SCHMIDT METHOD TEST

print('********** Running GS on A1...')
Q, R, Qqual, QRqual = mod_gramschmidt(A1)
print('')

print('********** Matrix Q of A1 **********')
print(Q)
print('')
print(f'********** "Orthogonal Quality" of Q: {Qqual}')
print('')

print('********** Matrix R of A1 **********')
print(R)
print('')
print(f'********** "Factorization Quality" of Q @ R: {QRqual}')
print('')

print('********** Running GS on Ab1...')
_, _, _, _ = mod_gramschmidt(Ab1)

********** Running GS on A1...

********** Matrix Q of A1 **********
[[ 0.49493563  0.27430061  0.67086781 -0.58353431]
 [ 0.10652533  0.02191507  0.71190709  0.59196435]
 [ 0.4727903   0.55434307 -0.11694396  0.52489343]
 [ 0.72122147  0.78548244 -0.17160648 -0.18317426]]

********** "Orthogonal Quality" of Q: 1.4134220179061285

********** Matrix R of A1 **********
[[1.09905346 0.         0.         0.        ]
 [0.         1.18225488 0.37028917 0.93376472]
 [0.         0.         0.9499728  0.6152959 ]
 [0.         0.         0.         0.66697111]]

********** "Factorization Quality" of Q @ R: 1.9455544728877086

********** Running GS on Ab1...
MATRIX IS NOT SQUARE!


## The Givens Method

The Givens (G) method takes into account the rotation matrices $G((h, k);X)$ that **set to zero** the $(h, k)$-th element of the matrix returned by the product $G((h,k);X) \, X$. Therefore, the Givens method consists in *setting to zero, one by one, all the elements under the diagonal* using the rotation matrices and, at the same time, obtaining an orthogonal matrix.

More precisely, given a square matrix $X\in\mathbb{R}^{n\times n}$, the method computes iteratively a matrix $Q$, through a product of rotation matrices, such that:
- $Q$ is orthogonal (i.e., $QQ^\top = \mathbb{I}_n = Q^{\top}Q$);
- The columns $\{\boldsymbol{q}_1,\ldots ,\boldsymbol{q}_n\}$ of $Q$ are an orthonormal base of the space $\langle \boldsymbol{x}_1,\ldots ,\boldsymbol{x}_n\rangle$ generated by the columns of $X$;
- $QX =: R\in\mathbb{R}^{n\times n}$ is an upper triangular matrix;

**N.B.:** if we want to preserve the same notation of the GS method, then G computes the matrix $Q^{\top}$ (such that $R=Q^\top X$ and, therefore, $QR=X$).

#### Rotation Matrices of the Givens Method (Case $h>k$)

The elements elements $g_{ij}:=G_{ij}((h,k); X)$ of a *Givens matrix* are defined as
$$
g_{ij}=
\begin{cases}
1\,,\quad & \text{if }i=j\neq h,k\\
c\,,\quad & \text{if }i=j= h,k\\
s\,,\quad & \text{if }i=k, j=h\\
-s\,,\quad & \text{if }i=h, j=k\\
0\,,\quad & \text{otherwise}\\
\end{cases}\,,
$$
where
$$
c:=\frac{x_{kk}}{\sqrt{x_{kk}^2 + x_{hk}^2}}\,,\quad s:=\frac{x_{hk}}{\sqrt{x_{kk}^2 + x_{hk}^2}}\,.
$$

Then, for the case $h>k$, the matrix has the following aspect:
$$
G((h,k);X) = 
\begin{bmatrix}
1 &    &    &    &    &    &    &    &    &    &    \\
    & \ddots &    &    &    &    &    &    &    &    &    \\
    & \quad & 1 &    &    &    &    &    &    &    &    &    \\
    &    &    & c & \cdots &    & \cdots & s &    &    &    \\
    &    &    & \vdots & 1 &    &    & \vdots &    &    &    \\
    &    &    &    &    & \ddots &    &    &    &    &    \\
    &    &    & \vdots &    &    & 1 & \vdots &    &    &    \\
    &    &    & -s & \cdots &    & \cdots & c &    &    &    \\
    &    &    &    &    &    &    &    & 1 &    &    &    \\
    &    &    &    &    &    &    &    &    & \ddots &    \\
    &    &    &    &    &    &    &    &    &    & 1 \\
\end{bmatrix}\,.
$$

#### Description of the G Method

From a "pseudocode point-of-view", given a square matrix $X\in\mathbb{R}^{n\times n}$, the algorithm performs the following operations:
1. $R\gets X$;
1. *initialization of* $Q$ (**exercise**)
1. **for** $k=1,\ldots ,n$ and $h=k+1, \ldots, n$ **do:** ($k$ for columns, $h$ for "under-diag." rows)
    1. $G\gets G((h, k); R)$
    1. *update* $Q$ (**exercise**)
    1. $R\gets GR$
1. **return** $Q$, $R$

#### Exercise 3: Givens Matrix

Complete the function in the following cell, such that it computes the Givens matrix $G((h,k); X)\in\mathbb{R}^{n\times n}$ for any square matrix $X\in\mathbb{R}^{n\times n}$ and any $h,k\in\{1,\ldots ,n\}$.

**Suggestions:** look at the help of the function np.**hypot** to compute the denominators of $c$ and $s$. Indeed, this function is better (from a numerical point of view) than using np.**sqrt** on the sum of $x_{kk}^2$ and $x_{hk}^2$.

In [6]:
def givens_mat(X, h, k):
    """
    Function that compute the Givens matrix for a given square matrix X and with respect to row h and column k
    :param X: square matrix represented as 2D-array object (numpy ndarray);
    :param h: integer value in the range of the number of X's rows;
    :param k: integer value in the range of the number of X's columns;
    :return G: the Givens matrix as 2D-array object (numpy ndarray).
    """
    
    m, n = X.shape
    
    if m != n:
        print('MATRIX IS NOT SQUARE!')
        return None
    
    # d (denominator of both c and s) can be written as:
    # d = np.sqrt(X[k, k]**2 + X[h, k]**2)
    # But is better (due to numerical problems) to use the hypot function.
    d = np.hypot(X[k,k], X[h,k]) 
    
    c = X[k,k] / d  
    s = X[h,k] / d 

    G = np.identity(n)
    G[h,h] = c
    G[k,k] = c
    G[h,k] = -s
    G[k,h] = s

    return G

In [7]:
# GIVENS MATRIX TEST

h, k = 1, 0
tolG = 1e-8

G_hk = givens_mat(A1, h, k)
G_hk_A1 = G_hk @ A1
G_hk_A1_tol = G_hk_A1.copy()
G_hk_A1_tol[abs(G_hk_A1_tol) < tolG] = 0

print('********** h, k **********')
print(f'h = {h} (row index)')
print(f'k = {k} (column index)')
print('')

print('********** Matrix G_hk **********')
print(G_hk)
print('')

print('********** Matrix G_hk @ A1 ((h,k)-th element should be zero) **********')
print('********** Original G_hk @ A1')
print(G_hk_A1)
print('')
print('********** "Rounded" G_hk @ A1')
print(G_hk_A1_tol)
print('')

********** h, k **********
h = 1 (row index)
k = 0 (column index)

********** Matrix G_hk **********
[[ 0.97761275  0.21041225  0.          0.        ]
 [-0.21041225  0.97761275  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]

********** Matrix G_hk @ A1 ((h,k)-th element should be zero) **********
********** Original G_hk @ A1
[[ 5.56417366e-01  3.22484817e-01  8.66342978e-01  4.53000912e-01]
 [ 2.07179288e-18 -4.29061024e-02  5.13616575e-01  7.75361423e-01]
 [ 5.19621810e-01  6.55374806e-01  9.41736558e-02  7.95759621e-01]
 [ 7.92660954e-01  9.28640444e-01  1.27834152e-01  5.05695092e-01]]

********** "Rounded" G_hk @ A1
[[ 0.55641737  0.32248482  0.86634298  0.45300091]
 [ 0.         -0.0429061   0.51361658  0.77536142]
 [ 0.51962181  0.65537481  0.09417366  0.79575962]
 [ 0.79266095  0.92864044  0.12783415  0.50569509]]



#### Exercise 4: Givens Method

Complete the function in the following cell, such that it performs the Givens method for any square matrix. The function must return not only $Q$ and $R$, but also the following norms representing the quality of the outputs:
- $||\mathbb{I}_n - QQ^\top||$;
- $||Q X - R||$.

**Suggestions:** exploit the **givens_mat** function of the previous exercise.

In [12]:
def givens(X):
    """
    Function that performs the Givens method for a given square matrix X.
    :param X: square matrix represented as 2D-array object (numpy ndarray);
    :return Q: 2D-array (orthogonal matrix) 
    :return R: 2D-array (upper triangle).
    :return Qqual: norm ||In - Q @ Q.T||
    :return QRqual: norm ||Q @ X - R||
    """
    
    m, n = X.shape
    
    if m != n:
        print('MATRIX IS NOT SQUARE!')
        return None, None, None, None
    
    # Initialization of the matrices
    R = X.copy()  # <-- TODO! 
    Q = np.identity(n)  # <-- TODO!
    
    for j in range(n):
        for i in range(j+1, m): 
            G = givens_mat(R, i, j)
            R = G@R
            Q = G@Q
            
            
    Qqual = np.linalg.norm(np.identity(n) - Q @ Q.T)  
    
    QRqual = np.linalg.norm(Q.T @ X - R)

    return Q, R, Qqual, QRqual

In [13]:
# GIVENS METHOD TEST

print('********** Running G on A1...')
print('')

Q, R, Qqual, QRqual = givens(A1)

tolR = 1e-8
Rtol = R.copy()
Rtol[abs(Rtol) < tolR] = 0



print('********** Matrix Q of A1 **********')
print(Q)
print('')
print(f'********** "Orthogonal Quality" of Q: {Qqual}')
print('')

print('********** Matrix R of A1 **********')
print('********** Original R')
print(R)
print('')
print('********** "Rounded" R')
print(Rtol)
print('')

print(f'********** "Factorization Quality" of Q.T @ R: {QRqual}')
print('')

print('********** Running G on Ab1...')
_, _, _, _ = givens(Ab1)

********** Running G on A1...

********** Matrix Q of A1 **********
[[ 0.49493563  0.10652533  0.4727903   0.72122147]
 [-0.79761806 -0.31671225  0.38015963  0.34493105]
 [-0.29391198  0.92355413  0.23079416 -0.08600913]
 [ 0.18016632 -0.18813153  0.76071154 -0.59452887]]

********** "Orthogonal Quality" of Q: 3.750255108769889e-16

********** Matrix R of A1 **********
********** Original R
[[ 1.09905346e+00  1.14287454e+00  5.75324249e-01  9.70286183e-01]
 [-1.08461185e-17  3.02596069e-01 -7.26206597e-01 -1.64161972e-02]
 [-2.09760250e-19 -2.57675522e-18  4.25662279e-01  8.46038352e-01]
 [-3.31897928e-18 -2.98207604e-18  1.51114628e-18  1.94551793e-01]]

********** "Rounded" R
[[ 1.09905346  1.14287454  0.57532425  0.97028618]
 [ 0.          0.30259607 -0.7262066  -0.0164162 ]
 [ 0.          0.          0.42566228  0.84603835]
 [ 0.          0.          0.          0.19455179]]

********** "Factorization Quality" of Q.T @ R: 2.9698558601370095

********** Running G on Ab1...
MATRIX IS

## The Householder Method

The Householder (H) method takes into account a family of reflection matrices such that, for any vector $\boldsymbol{x}\in\mathbb{R}^m$, the reflection matrix w.r.t. $\boldsymbol{x}$ is a square matrix $P_{\boldsymbol{x}}\in\mathbb{R}^{m\times m}$ such that
$$
P_{\boldsymbol{x}}\boldsymbol{x} = 
\begin{bmatrix}
-\sigma\\
0\\
\vdots\\
0
\end{bmatrix}\,,
$$
where $\sigma = \mathrm{sign}(x_1)||\boldsymbol{x}||$.

Then, with the same idea of the Givens method, iteratively we **set to zero** all the elements under the diagonal of the given input square matrix $X\in\mathbb{R}^{n\times n}$, one column by one column, using the reflection matrices and, at the same time, obtaining an orthogonal matrix $Q$.

More precisely, given a square matrix $X\in\mathbb{R}^{n\times n}$, the method computes iteratively a matrix $Q\in\mathbb{R}^{n\times n}$, through a product of matrices with a block that is a reflection matrix in $\mathbb{R}^{m\times m}$. Specifically, the method returns $Q, R\in\mathbb{R}^{n\times n}$ such that:
- $Q$ is orthogonal (i.e., $QQ^\top = \mathbb{I}_n = Q^{\top}Q$);
- The columns $\{\boldsymbol{q}_1,\ldots ,\boldsymbol{q}_n\}$ of $Q$ are an orthonormal base of the space $\langle \boldsymbol{x}_1,\ldots ,\boldsymbol{x}_n\rangle$ generated by the columns of $X$;
- $QX =: R\in\mathbb{R}^{n\times n}$ is an upper triangular matrix;

**N.B.:** if we want to preserve the same notation of the GS method, then H (as G) computes the matrix $Q^{\top}$ (such that $R=Q^\top X$ and, therefore, $QR=X$).

#### Reflection Matrices of the Householder Method

Given a vector $\boldsymbol{x}\in\mathbb{R}^m$ its Householder matrix $P_{\boldsymbol{x}}\in\mathbb{R}^{m\times m}$ is defined as
$$
P_{\boldsymbol{x}} = \mathbb{I}_m - 2 \bar{\boldsymbol{u}}\bar{\boldsymbol{u}}^\top\,,
$$
where 
$$
\bar{\boldsymbol{u}} = \frac{\boldsymbol{u}}{||\boldsymbol{u}||}
\quad \text{and}\quad
\boldsymbol{u}:= \boldsymbol{x} + \sigma\boldsymbol{e}_1 = \boldsymbol{x} + \mathrm{sign}(x_1)||\boldsymbol{x}||\boldsymbol{e}_1\,.
$$

**Few Details:** $P_{\boldsymbol{x}}\boldsymbol{x}$ is the reflection of the vector $\boldsymbol{x}$ w.r.t. the hyperplane orthogonal to the versor $\bar{\boldsymbol{u}}$.

#### Description of the H Method

From a "pseudocode point-of-view", given a square matrix $X\in\mathbb{R}^{n\times n}$, the algorithm performs the following operations:
1. $R\gets X$;
1. *initialization of* $Q$ (**exercise**)
1. **for** $j=1,\ldots ,n$ **do:** ($j$ for columns)
    1. $\boldsymbol{x}\gets R_{j:n, j}$ (sub-column of $j$-th column, elements "from the diagonal to the end")
    1. $P_j\gets \begin{bmatrix} \mathbb{I}_{n-j} & \boldsymbol{0} \\ \boldsymbol{0} & P_{\boldsymbol{x}}\end{bmatrix}$ 
    1. *update* $Q$ (**exercise**)
    1. $R\gets P_jR$
1. **return** $Q$, $R$

#### Exercise 5: Householder Matrix

Complete the function in the following cell, such that it computes the Housholder matrix $P_{\boldsymbol{x}}\in\mathbb{R}^{m\times m}$ for any vector $\boldsymbol{x}\in\mathbb{R}^{m}$.

**Suggestions:** look at the help of the function np.**sign** to compute $\sigma$.

In [10]:
def householder_mat(x):
    """
    Function that compute the reflection matrix of the Householder method for a given vector x.
    :param x: vector that can be given both as 1D-array object and 2D-array column/row object (numpy ndarray);
    :return Px: Householder reflection matrix as 2D-array object (numpy ndarray).
    """
    
    # Reshaping the input as a column vector (if it is 1D-array or row 2D-array)... actually it works even if it is a non-vectot matrix...
    v = x.reshape(x.size, 1)

    sigma = ...  # <-- TODO!

    # Computation of u (versor)
    ...  # <-- TODO!

    # Computation of the reflection matrix
    ...  # <-- TODO!

    return Px

In [11]:
# HOUSEHOLDER MATRIX TEST

tolH = 1e-8

Pb1 = householder_mat(b1)
Pb1b1 = Pb1 @ b1
Pb1b1_tol = Pb1b1.copy()
Pb1b1_tol[abs(Pb1b1_tol) < tolH] = 0

print('********** Matrix Pb1 **********')
print(Pb1)
print('')

print('********** Matrix Pb1 @ b1 **********')
print('********** Original Pb1 @ b1')
print(Pb1b1)
print('')
print('********** "Rounded" Pb1 @ b1')
print(Pb1b1_tol)
print('')

NameError: name 'Px' is not defined

#### Exercise 4: Householder Method

Complete the function in the following cell, such that it performs the Householder method for any square matrix. The function must return not only $Q$ and $R$, but also the following norms representing the quality of the outputs:
- $||\mathbb{I}_n - QQ^\top||$;
- $||Q X - R||$.

**Suggestions:** exploit the **householder_mat** function of the previous exercise.


In [ ]:
def householder(X):
    """
    Function that performs the Householder method for a given square matrix X.
    :param X: square matrix represented as 2D-array object (numpy ndarray);
    :return Q: 2D-array (orthogonal matrix) 
    :return R: 2D-array (upper triangle).
    :return Qqual: norm ||In - Q @ Q.T||
    :return QRqual: norm ||Q @ X - R||
    """
    
    m, n = X.shape
    
    if m != n:
        print('MATRIX IS NOT SQUARE!')
        return None, None, None, None
    
    # Initialization of the matrices
    R = ...  # <-- TODO!
    Q = ...  # <-- TODO!

    for j in range(n):
        ...  # <-- TODO!
    
    Qqual = ...  # <-- TODO!
    
    QRqual = ...  # <-- TODO!

    return Q, R, Qqual, QRqual

In [ ]:
# HOUSEHOLDER METHOD TEST

print('********** Running H on A1...')
print('')

Q, R, Qqual, QRqual = householder(A1)

tolR = 1e-8
Rtol = R.copy()
Rtol[abs(Rtol) < tolR] = 0



print('********** Matrix Q of A1 **********')
print(Q)
print('')
print(f'********** "Orthogonal Quality" of Q: {Qqual}')
print('')

print('********** Matrix R of A1 **********')
print('********** Original R')
print(R)
print('')
print('********** "Rounded" R')
print(Rtol)
print('')

print(f'********** "Factorization Quality" of Q.T @ R: {QRqual}')
print('')

print('********** Running H on Ab1...')
_, _, _, _ = householder(Ab1)

## A Comparison Between Methods


Let $V_n(\boldsymbol{x})\in\mathbb{R}^{m\times n}$ be the *Vandermonde* matrix of order $n$ w.r.t. the vector $\boldsymbol{x}\in\mathbb{R}^m$, i.e.:
$$
V_n(\boldsymbol{x}) = 
\begin{bmatrix}
x_1^0 & \cdots & x_1^{n-1} \\
\vdots &  & \vdots \\
x_m^0 & \cdots & x_m^{n-1} \\
\end{bmatrix}
$$

Considering only square Vandermonde matrices (i.e., $m=n$), study how the quality of the orthogonalization changes while increasing $n$ (and, consequently, the condition number $\kappa(V_n(\boldsymbol{x}))$). 

In particular, look at the quality measures returned by the methods and the corresponding plots.

#### Exercise 5: Methods and Performances

Complete the code in the following cell, such that it computes and illustrates how the norms $||\mathbb{I}_n - QQ^\top||$ and $||Q X - R||$ (or $||Q^{\top} X - R||$) changes with $n$

**Suggestions:** use the **vander** function for computing the Vandermonde matrix.


In [ ]:
n_values = [2 ** i for i in range(8)]

x = 2 * np.random.rand(n_values[-1])  # Uniform random values in [0, 2]

Qqual_df = pd.DataFrame(np.zeros((len(n_values), 4)), columns=['GS', 'mGS', 'G', 'H'], index=n_values)
QRqual_df = pd.DataFrame(np.zeros((len(n_values), 4)), columns=['GS', 'mGS', 'G', 'H'], index=n_values)

cond_values = []

for n in n_values:
    # Select the first n elements of x for building the order-n Vandermonde matrix
    xn = ...  # <-- TODO!
    V = np.vander(..., increasing=True)  # <-- TODO!
    
    cond_values.append(np.linalg.cond(V))

    # Compute the "quality norms" of the algorithms and store the results in the pandas DataFrame.
    # EXAMPLE for storing value in the pandas DataFrame for the case of GS method ("Qqual" norm):
    #     Qqual_df.loc[n, 'GS'] = Qqual_norm_GS
    #
    # ... <-- TODO!
    

print('******* "ORTHOGONAL QUALITY" OF Q *******')
display(Qqual_df)
print('')

print('******* "FACTORIZATION QUALITY" *******')
display(QRqual_df)
print('')


# ***************** FIGURE 0 *****************
fig0, ax0 = plt.subplots(1, 2, figsize=(10,5))
ax0[0].plot(n_values, cond_values, '-*', label='cond. num.')

ax0[1].plot(n_values, cond_values, '-*', label='cond. num.')

ax0[0].set_title('')
ax0[0].set_xlabel('order (n)')
ax0[0].set_ylabel('cond. num.')
ax0[0].grid()
ax0[0].legend()

ax0[1].set_title('LOG. SCALE')
ax0[1].set_xlabel('order (n)')
ax0[1].set_ylabel('cond. num.')
ax0[1].set_yscale(...)  # <-- TODO!
ax0[1].grid()
ax0[1].legend()

fig0.suptitle('V_n(x) Condition Number') 


# ***************** FIGURE 1 *****************
fig1, ax1 = plt.subplots(1, 2, figsize=(10, 5))

ax1[0].plot(..., '--x', label=...)  # <-- TODO!
...  # <-- TODO!


ax1[1].plot(..., '--x', label=...) # <-- TODO!
...  # <-- TODO!

ax1[0].set_title('||I_n - Q @ Q.T||')
ax1[0].set_xlabel('order (n)')
ax1[0].set_ylabel('norm value')
ax1[0].grid()
ax1[0].legend()

ax1[1].set_title('||Q @ X - R|| or ||Q.T @ X - R||')
ax1[1].set_xlabel('order (n)')
ax1[1].set_ylabel('norm value')
ax1[1].grid()
ax1[1].legend()

fig1.suptitle('ORTHOGONALIZATION QUALITY') 


# ***************** FIGURE 2 *****************
fig2, ax2 = plt.subplots(1, 2, figsize=(10, 5))

ax2[0].plot(..., '--x', label=...)  # <-- TODO!
...  # <-- TODO!


ax2[1].plot(..., '--x', label=...) # <-- TODO!
...  # <-- TODO!

ax2[0].set_title('||I_n - Q @ Q.T||')
ax2[0].set_xlabel('order (n)')
ax2[0].set_ylabel('norm value')
ax2[0].set_yscale(...)  # <-- TODO!
ax2[0].grid()
ax2[0].legend()

ax2[1].set_title('||Q @ X - R|| or ||Q.T @ X - R||')
ax2[1].set_xlabel('order (n)')
ax2[1].set_ylabel('norm value')
ax2[1].set_yscale(...)  # <-- TODO!
ax2[1].grid()
ax2[1].legend()

fig2.suptitle('ORTHOGONALIZATION QUALITY (LOG. SCALE)') 
    
    